In [2]:
!gdown --id 1vkb2aGaHSkdmQk4S1CQYMvuFR28_HJ2b

In [ ]:
!unzip /content/Fonts.zip

#Library

In [4]:
import torch
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

In [5]:
dataset_folder = '/content/Fonts'
transform = transforms.Compose([transforms.ToTensor()])
dataset = torchvision.datasets.ImageFolder(root = dataset_folder, transform = transform) 

In [6]:
len(dataset)

372450

In [7]:
dataset.class_to_idx

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [8]:
classes = dataset.classes
classes

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

#Split dataset

In [9]:
split_ratio = 0.75 
train_set, val_set = torch.utils.data.random_split(dataset, [int(len(dataset)*split_ratio), len(dataset)- int(len(dataset)*split_ratio)])

In [10]:
batch = 2048

In [11]:
trainloader = torch.utils.data.DataLoader(train_set, batch_size = batch, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(val_set, batch_size = batch, shuffle=True, num_workers=2,drop_last= True)

#Vizuals

In [12]:
def imshow(img):
    img = img     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(batch)))

#model

In [ ]:
model = torchvision.models.shufflenet_v2_x0_5(pretrained= True)
model.fc = nn.Linear(1024 , len(classes))
model

#Training

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cuda', index=0)

In [16]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [17]:
step = 0
loss_train = []
loss_val = []

#import the model

model = model.to(device)
model = model.train()

min_loss = 100
patience = 5
training_loss_store = []
validation_loss_store = []

In [18]:
len(trainloader)
2048 * 137

280576

In [19]:
file = open('logs_test4_epoch100_with_max_pool.txt', 'w')

print('training started.............................................')
file.write('training started.............................................\n')
start_time = time.time()
for epoch in range(50):  # loop over the dataset multiple times
    #file.write('##############################TRAINING###############################\n')
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        step+=1
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device),data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss_train.append(loss.item())
        training_loss_store.append([epoch, loss.item()])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.5f' %(epoch + 1, i + 1, running_loss / 2000))
            #log_loss_summary(logger, loss_train, step, prefix = 'train_')
            file.write('epoch = '+ str(epoch + 1) + '\t' +'step = '+ str(step) +'\t'+'train_loss = '+'\t'+str(np.mean(loss_train)) +'\n')
            loss_train = []
            running_loss = 0.0
            
    print('Finished training for epoch ' + str(epoch) + ' time taken = ' + str(time.time() - start_time))
    file.write('Finished training for epoch ' + str(epoch) + ' time taken = ' + str(time.time() - start_time) + '\n')
    file.write('##################################evaluation##############################\n')
    print('################################evaluation###########################\n')
    with torch.no_grad():
        val_loss = 0
        model.eval()
        
        for i, data in enumerate(testloader, 0):
            step+=1
            inputs, labels = data[0].to(device),data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss_val.append(loss.item())
            validation_loss_store.append([epoch, loss.item()])
            val_loss += loss
            
        val_loss = val_loss/float(i + 1)
        
        if val_loss < min_loss:
            min_loss = val_loss
            no_impr_epoch = 0
            
            #save the best model
            #torch.save(model.state_dict(), 'weight/' + 'epoch_' + str(epoch+1) + 'loss_' + str(val_loss) + '.pt')
            print(str(val_loss))
            torch.save(model.state_dict(), 'xyz_epoch_'+str(epoch+1)+'.pt')
            
            print('performance improved with validation loss ' + str(val_loss))
            file.write('--------------------------------------------------------------------\n')
            file.write('performance improved with validation loss =  ' + str(val_loss) + '\n')
            
            file.write('epoch = '+ str(epoch + 1) + '\t' +'step = '+ str(step) +'\t'+'val_loss = '+'\t'+str(np.mean(loss_val)) +'\n')
            file.write('--------------------------------------------------------------------\n\n')
            #log_loss_summary(logger, loss_val, step, prefix="val_")
            loss_val = []
        else:
            no_impr_epoch += 1
            print('no improvement with prev best model ' + str(no_impr_epoch) + 'th')
            file.write('no improvement with prev best model ' + str(no_impr_epoch) + 'th \n')
            
        if no_impr_epoch > patience:
            print('stop training')
            file.write('stop training')
            break
    
print('Finished Training................................................')
file.write('Finished Training................................................\n')
end_time = time.time()
file.write('Training time:- ' + str(end_time - start_time))
file.close()

training started.............................................


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,    20] loss: 0.03263
[1,    40] loss: 0.03253
[1,    60] loss: 0.03240
[1,    80] loss: 0.03227
[1,   100] loss: 0.03215
[1,   120] loss: 0.03202
Finished training for epoch 0 time taken = 57.24110412597656
################################evaluation###########################

tensor(3.1821, device='cuda:0')
performance improved with validation loss tensor(3.1821, device='cuda:0')
[2,    20] loss: 0.03031
[2,    40] loss: 0.02584
[2,    60] loss: 0.02171
[2,    80] loss: 0.01708
[2,   100] loss: 0.01253
[2,   120] loss: 0.00949
Finished training for epoch 1 time taken = 137.58702087402344
################################evaluation###########################

tensor(0.7267, device='cuda:0')
performance improved with validation loss tensor(0.7267, device='cuda:0')
[3,    20] loss: 0.00704
[3,    40] loss: 0.00620
[3,    60] loss: 0.00543
[3,    80] loss: 0.00516
[3,   100] loss: 0.00483
[3,   120] loss: 0.00443
Finished training for epoch 2 time taken = 217.92687678337097
###########

In [ ]:
PATH = 'xyz_epoch_50.pt'
model.load_state_dict(torch.load(PATH))
model.eval()

#Testing

In [30]:
dataiter = iter(testloader)
images, labels = dataiter.next()

In [31]:
print('Ground Truth: ', ' '.join('%5s' % classes[labels[j]] for j in range(batch)))

Ground Truth:      S     W     S     L     N     B     Q     L     U     U     M     K     S     C     O     T     S     T     W     S     N     T     N     L     N     P     E     S     S     B     O     O     S     S     O     D     S     S     P     Q     A     Z     S     L     G     N     C     L     C     C     N     U     N     R     U     S     G     D     S     O     S     S     U     W     L     E     A     R     S     A     C     T     A     A     D     U     A     Y     S     C     B     S     M     O     P     D     G     P     U     S     H     H     C     C     O     B     O     T     C     H     S     S     M     N     M     B     N     S     W     Y     A     N     K     C     S     K     O     S     S     P     Q     P     N     T     B     T     E     V     N     A     A     Z     S     M     O     C     S     S     O     S     W     P     S     J     R     U     M     O     U     E     V     X     C     B     O     G     W     B     E     O     P     M     S     S  

In [32]:
outputs = model(images.to(device))
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(batch)))

Predicted:      S     W     S     L     N     B     Q     L     U     U     M     K     S     C     O     T     S     T     W     S     N     T     N     L     N     P     E     S     S     B     O     O     S     S     O     D     S     S     P     Q     N     Z     S     L     G     N     C     L     C     C     N     U     N     R     U     S     G     B     S     O     S     S     U     W     L     E     A     R     S     A     C     T     A     A     D     U     A     Y     S     C     B     S     M     O     P     D     G     P     U     S     H     H     C     C     O     B     O     T     C     H     S     J     M     N     M     B     N     S     W     Y     A     N     K     C     S     K     O     S     S     P     Q     P     N     T     B     T     E     V     N     M     A     Z     S     M     O     C     S     S     O     S     W     P     S     J     R     U     M     O     U     E     V     X     C     B     O     G     W     B     E     O     P     M     S     S     

In [33]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 3776 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 3776 test images: 97 %


In [34]:
class_correct = list(0. for i in range(len(classes)))
class_total = list(0. for i in range(len(classes)))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(batch):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(len(classes)):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     A : 96 %
Accuracy of     B : 95 %
Accuracy of     C : 97 %
Accuracy of     D : 92 %
Accuracy of     E : 96 %
Accuracy of     F : 93 %
Accuracy of     G : 87 %
Accuracy of     H : 90 %
Accuracy of     I : 95 %
Accuracy of     J : 96 %
Accuracy of     K : 94 %
Accuracy of     L : 97 %
Accuracy of     M : 97 %
Accuracy of     N : 97 %
Accuracy of     O : 98 %
Accuracy of     P : 98 %
Accuracy of     Q : 93 %
Accuracy of     R : 95 %
Accuracy of     S : 99 %
Accuracy of     T : 98 %
Accuracy of     U : 97 %
Accuracy of     V : 97 %
Accuracy of     W : 94 %
Accuracy of     X : 95 %
Accuracy of     Y : 96 %
Accuracy of     Z : 98 %


In [26]:
import os
folders = os.listdir("Fonts")

In [ ]:
folders

In [ ]:
for i in folders :
  print(i,len(os.listdir(os.path.join("Fonts",i))))